In [14]:
import pandas as pd
import numpy as np
import re
import os
import time
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import nltk
import faiss

In [15]:
!pip install faiss-cpu

In [16]:
file_path = "Corpus_completo.xlsx"
corpus_completo = pd.read_excel(file_path)


In [17]:
# Formatear fechas

meses = {
    'enero': '01',
    'febrero': '02',
    'marzo': '03',
    'abril': '04',
    'mayo': '05',
    'junio': '06',
    'julio': '07',
    'agosto': '08',
    'septiembre': '09',
    'octubre': '10',
    'noviembre': '11',
    'diciembre': '12'
}

def convertir_fecha(fecha_str):
    partes = fecha_str.split(' de ')
    dia = partes[0].zfill(2)  # Asegura 2 dígitos para días 1-9
    mes = meses[partes[1]]
    año = partes[2]
    return f"{dia}/{mes}/{año}"  # Formato DD/MM/AAAA

corpus_completo['Fecha_formateada'] = corpus_completo['Fecha'].apply(convertir_fecha)
corpus_completo['Fecha_formateada'] = pd.to_datetime(corpus_completo['Fecha_formateada'], format='%d/%m/%Y', errors='coerce')

In [18]:
corpus_completo

,Diario,Autor,Fecha,Título,Texto,Vínculo,ID,Fecha_formateada
0,El Espectador,Gonzalo Hernández,1 de enero de 2018,Fajardo: para nada tibio,"La Coalición Colombia Partido Alianza Verde, ...",https://web.archive.org/web/20180102104221/htt...,1,2018-01-01
1,El Espectador,Eduardo Barajas Sandoval,1 de enero de 2018,Macedonia de Norte,Las interpretaciones de la historia sirven com...,https://web.archive.org/web/20180102104221/htt...,2,2018-01-01
2,El Espectador,Daniel Emilio Rojas Castro,1 de enero de 2018,El nacionalismo según Vargas Llosa,La semana pasada Mario Vargas Llosa publicó un...,https://web.archive.org/web/20180102104221/htt...,3,2018-01-01
3,El Espectador,Reinaldo Spitaletta,1 de enero de 2018,"Tiempo sagrado, tiempo profano","Pudiera decirse, sin ser una verdad absoluta, ...",https://web.archive.org/web/20180102104221/htt...,4,2018-01-01
4,El Espectador,Aura Lucía Mera,1 de enero de 2018,La rebelión de los bueyes,Lo mejor del encierro de Las Ventas fueron los...,https://web.archive.org/web/20180102104221/htt...,5,2018-01-01
...,...,...,...,...,...,...,...,...
13335,Semana,Esteban Piedrahita,25 de diciembre de 2018,Capitalismo a la escandinava,Los países nórdicos coquetearon con el sociali...,https://web.archive.org/web/20181227221015/htt...,13336,2018-12-25
13336,Semana,Germán Manga,26 de diciembre de 2018,"Ahora, a derrotar a los Guachos del Amazonas",Solo una acción militar sostenida y de gran es...,https://web.archive.org/web/20181227220937/htt...,13337,2018-12-26
13337,Semana,Ariel Ávila,26 de diciembre de 2018,Porte de armas y Estado de derecho,El debate del porte de armas es más complejo d...,https://web.archive.org/web/20181227220928/htt...,13338,2018-12-26
13338,Semana,Lucas Pombo,27 de diciembre de 2018,Quién es quién en el Congreso,"En estos primeros meses de actividad, algunos ...",https://web.archive.org/web/20181227220919/htt...,13339,2018-12-27


In [19]:
# Generar embeddings de textos
modelo_embedding = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')


In [40]:
# elegir  3000 columnas aleatoriamente
corpus_filtro = corpus_completo.sample(n=3000, random_state=42)
textos = corpus_filtro['Texto'].astype(str).tolist()  # Convertir a lista de strings
#textos = corpus_completo['Texto'][:3000].astype(str).tolist()  
embeddings = modelo_embedding.encode(textos, convert_to_tensor=False).astype('float32')


In [46]:
# Crear índice FAISS
indice = faiss.IndexFlatL2(embeddings.shape[1])
indice.add(embeddings)

In [67]:
# Buscar texto similar a una consulta
consulta = ["gabriel garcia marquez"]
embedding_consulta = modelo_embedding.encode(consulta, convert_to_tensor=False).astype('float32')
distancias, indices = indice.search(embedding_consulta, k=20)

In [68]:
distancias

array([[3.9210205, 4.084386 , 4.199977 , 4.268858 , 4.376219 , 4.415838 ,
        4.4358063, 4.4642763, 4.4694195, 4.52221  , 4.5312476, 4.579609 ,
        4.5844107, 4.590514 , 4.591217 , 4.655558 , 4.6766205, 4.7154617,
        4.728218 , 4.7378626]], dtype=float32)

In [69]:
indices[0]  # Índices de los textos más similares

array([ 316,  640, 2918, 2760, 2002, 2007, 1694, 1723, 2994, 2215, 2542,
        702, 2851, 1045,  404, 2596,  842,  729, 1385, 2591], dtype=int64)

In [55]:
# Filtrar la fila

print(corpus_filtro.iloc[33])

Diario                                                      El Tiempo
Autor                                         Óscar Domínguez Giraldo
Fecha                                             19 de junio de 2020
Título                                             Cargando gladiolos
Texto               El nobel García Márquez habría muerto del sust...
Vínculo             https://web.archive.org/web/20200627013433/htt...
ID                                                              12158
Fecha_formateada                                  2020-06-19 00:00:00
Name: 12157, dtype: object


In [71]:
#Resultados
print("Textos más similares:")
for idx in indices[0]:
    print(f"Índice: {idx}")
    print("Texto: " + textos[idx])
    print("---------------------------------------")


Textos más similares:
Índice: 316
Texto: De Carrasquilla se ha dicho que está pasado de moda; que su lenguaje es indescifrable; que no interpreta a la nación colombiana. En fin, que es un meapilas sin importancia. Por supuesto me refiero a don Tomás, no a don Alberto Carrasquilla, quien deberá dedicar toda su experimentada energía a los duros desafíos que vienen.

A Tomás Carrasquilla Naranjo se le otorgó tarde el Premio Nacional de Literatura, en 1936. Paradójicamente, la resolución que lo otorga va firmada por Alfonso López Pumarejo y Darío Echandía. El jurado también era liberal, compuesto por Baldomero Sanín Cano y Jorge Zalamea. Tratan de equilibrar la ecuación política Antonio Gómez Restrepo, secretario general de la Cancillería por más de veinte años, y Laureano García Ortiz, presidente de la Academia y canciller de Suárez.

Todos estos nombres parecen impulsados por la fatalidad de Rodó: La que nos empuja a la política a todos los que tenemos una pluma en la mano. A don Tomás